In [6]:
#示例：langgraph_hello.py
import os
from dotenv import load_dotenv
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_community.chat_models import ChatZhipuAI
# pip install langgraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import requests
import json


# 加载 .env 文件中的环境变量
load_dotenv(dotenv_path='D:/02File/05练习项目/04llm_learning/KEYs.env', override=True)
# 获取 API 密钥
llm_api_key = os.environ["ZHIPUAI_API_KEY"] 

llm = ChatZhipuAI(
    model="GLM-4-Air",
    api_key=llm_api_key,
    temperature=0
)

weather_api_key = os.environ["WEATHER_API_KEY"]

def city_code_search(city_name: str) -> str:
    """
    输入城市名称，返回城市代码。
    """
    # 打开并读取JSON文件
    with open("D:/02File/05练习项目/04llm_learning/month1/week1/langgraph/tools/city_code.json", 'r', encoding='utf-8') as file:
        data = json.load(file)
    # 访问数据
    for province in data['list']:
        for city in province['list']:
            if city['name'] == city_name:
                return city['city_id']
    return None
# 定义工具函数，用于代理调用外部工具
@tool
def search(city_name: str) -> str:
    """
    输入城市名称，获取天气信息。
    """
    city_code = city_code_search(city_name)
    if city_code:
        # API 地址
        url = "http://api.yytianqi.com/observe"
        # 可选的查询参数
        params = {
            "city": city_code,
            "key": weather_api_key
        }
        # 发送 GET 请求
        response = requests.get(url, params=params)
        # 检查响应状态码
        if response.status_code == 200:
            # 解析返回的数据（假设是 JSON 格式）
            data = response.json()
            # print(f"API Response Data: {data}")
            tq = data['data']['tq']
            qw = data['data']['qw']
            return f"查询城市当前天气{tq}，气温{qw}度。"
        else:
            print(f"Error: {response.status_code}")   
    else:
        return "不支持查询该城市的天气"

In [18]:
# 将工具函数放入工具列表
tools = [search]

# 创建工具节点
tool_node = ToolNode(tools)

# 1.初始化模型和工具，定义并绑定工具到模型
model = llm.bind_tools(tools)

In [19]:
# 定义函数，决定是否继续执行
def should_continue(state: MessagesState) -> Literal["tools", END]: # type: ignore
    messages = state['messages']
    last_message = messages[-1]
    # 如果LLM调用了工具，则转到“tools”节点
    if last_message.tool_calls:
        return "tools"
    # 否则，停止（回复用户）
    return END


# 定义调用模型的函数
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # 返回列表，因为这将被添加到现有列表中
    return {"messages": [response]}

In [20]:
# 2.用状态初始化图，定义一个新的状态图
workflow = StateGraph(MessagesState)
# 3.定义图节点，定义我们将循环的两个节点
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# 4.定义入口点和图边
# 设置入口点为“agent”
# 这意味着这是第一个被调用的节点
workflow.set_entry_point("agent")

In [21]:
# 添加条件边
workflow.add_conditional_edges(
    # 首先，定义起始节点。我们使用`agent`。
    # 这意味着这些边是在调用`agent`节点后采取的。
    "agent",
    # 接下来，传递决定下一个调用节点的函数。
    should_continue,
)

# 添加从`tools`到`agent`的普通边。
# 这意味着在调用`tools`后，接下来调用`agent`节点。
workflow.add_edge("tools", 'agent')

# 初始化内存以在图运行之间持久化状态
checkpointer = MemorySaver()

In [22]:
# 5.编译图
# 这将其编译成一个LangChain可运行对象，
# 这意味着你可以像使用其他可运行对象一样使用它。
# 注意，我们（可选地）在编译图时传递内存
app = workflow.compile(checkpointer=checkpointer)
# app = workflow.compile()

In [23]:
# 6.执行图，使用可运行对象
final_state = app.invoke(
    # {"messages": [HumanMessage(content="北京（城市代码为CH010100）和上海（城市代码为CH020100）的天气怎么样?")]},
    {"messages": [HumanMessage(content="大连的天气怎么样?")]},
    # {"messages": [HumanMessage(content="北京的城市代码是多少?")]},
    config={"configurable": {"thread_id": 42}}
)
# 从 final_state 中获取最后一条消息的内容
result = final_state["messages"][-1].content
print(result)
print(final_state)

很抱歉，看起来在尝试获取大连天气信息时发生了错误。我将尝试重新获取正确的信息。请稍等，我将重新查询大连的天气情况。
{'messages': [HumanMessage(content='大连的天气怎么样?', additional_kwargs={}, response_metadata={}, id='8e81dd4a-6407-4adb-8a57-e47f51a790d0'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"city_name": "大连"}', 'name': 'search'}, 'id': 'call_-8724721522099433855', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 151, 'total_tokens': 161}, 'model_name': 'GLM-4-Air', 'finish_reason': 'tool_calls'}, id='run-0d5bde85-2ed9-4257-aef6-ec4f5a1dea59-0', tool_calls=[{'name': 'search', 'args': {'city_name': '大连'}, 'id': 'call_-8724721522099433855', 'type': 'tool_call'}]), ToolMessage(content='Error: TypeError("\'NoneType\' object is not subscriptable")\n Please fix your mistakes.', name='search', id='22916b0d-33a2-4e97-acb5-15c22f6f3e5b', tool_call_id='call_-8724721522099433855', status='error'), AIMessage(content='', additional_kwargs={'tool_call

In [13]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="我问的哪个城市?")]},
    config={"configurable": {"thread_id": 42}}
)
result = final_state["messages"][-1].content
print(result)
print(final_state)

您问的是北京的天气情况。我之前的回答是基于您的问题“北京的天气怎么样?”所提供的假设性信息。如果您需要查询北京或其他城市的实时天气信息，您可以使用天气应用程序或者访问天气网站来获取最新的天气数据。
{'messages': [HumanMessage(content='北京的天气怎么样?', additional_kwargs={}, response_metadata={}, id='12839bf7-e2ba-416e-add3-4ee1bc676323'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"city_name": "北京"}', 'name': 'search'}, 'id': 'call_-8724721350300714677', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 150, 'total_tokens': 160}, 'model_name': 'GLM-4-Air', 'finish_reason': 'tool_calls'}, id='run-0efe0381-1dda-4963-9df7-61673e9ccfa8-0', tool_calls=[{'name': 'search', 'args': {'city_name': '北京'}, 'id': 'call_-8724721350300714677', 'type': 'tool_call'}]), ToolMessage(content='查询城市当前天气晴，气温28度。', name='search', id='9b6526e5-b9f9-49ab-a694-81ebbc076165', tool_call_id='call_-8724721350300714677'), AIMessage(content='很抱歉，我无法直接提供实时天气信息，因为我没有直接访问天气数据库的能力。但是，根据您提供的信息，北京现在的天气是晴朗的，气温为28摄氏度。如果您需要更准确的天